GroupLens 提供各種不同大小的影評檔案給大家測試，避免執行時間過長，我們選擇最小的檔案ml-100k.zip測試，其中README檔案有詳細敘述每一個資料檔的用途與欄位，這裡會用到的檔案說明如下：

u.data：包含 943 個使用者對 1682 部電影所做的 100000 筆評論，欄位包括：

- 使用者代碼(user id)
- 電影代碼(item id)
- 評論分數(rating)
- 時間戳記(timestamp)

u.item：1682 部電影基本資料，欄位很多，這裡只會用到前兩欄，電影代碼(movie id)、電影名稱(movie title)。
- 處理步驟如下：

1. 讀取u.item、u.data 兩個檔案，並合併。
2. 使用樞紐分析函數(pivot_table)，將資料轉換 USER-ITEM Matrix。
3. 計算 USER-USER、ITEM-ITEM Similarity Matrix。
4. 隨機指定一個使用者或商品，進行測試，列出推薦的商品。
5. 資料轉換

In [13]:
import pandas as pd

In [14]:
# item是一個特別的資料型態，這邊打開原始資料給大家做示範
# Read the input training data
input_data_file_movie = "ml-100k/u.item"
input_data_file_rating = "ml-100k/u.data"

movie = pd.read_csv(input_data_file_movie, sep='|', encoding='ISO-8859-1', names=['movie_id', 'movie_title'], usecols = [0,1]) 
#前兩個feature設為'movie_id', 'movie_title'  sep='|'

rating = pd.read_csv(input_data_file_rating, sep='\t', encoding='ISO-8859-1', names=["user_id","movie_id","rating"], usecols = [0,1,2])
#前三個feature設為"user_id","movie_id","rating"， sep='\t'
print(movie.head())
print(rating.head())

   movie_id        movie_title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)
   user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1


In [15]:
# then merge movie and rating data
data = pd.merge(movie,rating)
data.head()

,movie_id,movie_title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


使用樞紐分析函數(pivot_table)，將資料轉換 USER-ITEM Matrix。

P.S. pivot_table為pandas的內建方法。
- 使用方法如下:
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)

常用參數：

- data : 讀取你要使用的 DataFrame
- index : 必要參數。此處輸入不想要變動的數據，作為想要比較的欄位基礎，該數據會成為第一欄的索引（index），此處能以 list、array 等方式輸入多個 index，則結果會以巢狀的方式呈現。
- values : 可選。可以對需要計算的數據做篩選，如果以 list、array 等方式輸入多個 value，則能夠分別獲得該欄位的不同數值。
- columns : 可選。用以分割數據，去選出想比較的特定欄位。
- aggfunc : function 參數。是 Pivot Table 裡最厲害的功能，能夠引入 max、min 等內建參數，甚至能自訂 function 使用。

選用參數：

- fill_value : 用特定值取代 NULL 的欄位。
- margins : 布林值，用來確認是否顯示該欄位的加總。
- margins_name : 字串，用來顯示上面 margin 增加的列或欄的名稱。
- dropna : 布林值， 用以丟棄缺失值。
- observed : 布林值，當 grouper 為 Categoricals 使用。

In [16]:
type(rating)

pandas.core.frame.DataFrame

USER-ITEM Matrix
--

In [17]:
pivot_table = data.pivot_table(index = ["user_id"],columns = ["movie_title"],values = "rating")
pivot_table.head(10)

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
6,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,...,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pivot_table["Toy Story (1995)"]

user_id
1      5.0
2      4.0
3      NaN
4      NaN
5      4.0
      ... 
939    NaN
940    NaN
941    5.0
942    NaN
943    NaN
Name: Toy Story (1995), Length: 943, dtype: float64

ITEM-ITEM 協同過濾相似性(Similarity)計算
--
使用關聯係數函數(corrwith)，計算 ITEM-ITEM 協同過濾相似性(Similarity)。得到與 Bad Boys (1995) 這部電影最相似的5部電影名稱。

In [19]:
movie_watched = pivot_table["Bad Boys (1995)"]
similarity_with_other_movies = pivot_table.corrwith(movie_watched)  # find correlation between "Bad Boys (1995)" and other movies
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head()

C:\Users\9mzx8\anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\9mzx8\anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


movie_title
So Dear to My Heart (1949)                          1.0
Gold Diggers: The Secret of Bear Mountain (1995)    1.0
Simple Twist of Fate, A (1994)                      1.0
Germinal (1993)                                     1.0
American Dream (1990)                               1.0
dtype: float64

修改上述兩步驟，計算 USER-USER 協同過濾相似性(Similarity)。得到第10個使用者最相似的使用者，再根據他們的喜好推薦。

In [20]:
# lets make a pivot table in order to make rows are users and columns are movies. And values are rating
pivot_table = data.pivot_table(index =["movie_title"],columns =  ["user_id"],values = "rating")
print(pivot_table.head(10))
target_user = pivot_table[10]
similarity_with_other_movies = pivot_table.corrwith(target_user)  # find correlation between "Bad Boys (1995)" and other movies
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head()

user_id                                      1    2    3    4    5    6    \
movie_title                                                                 
'Til There Was You (1997)                    NaN  NaN  NaN  NaN  NaN  NaN   
1-900 (1994)                                 NaN  NaN  NaN  NaN  NaN  NaN   
101 Dalmatians (1996)                        2.0  NaN  NaN  NaN  2.0  NaN   
12 Angry Men (1957)                          5.0  NaN  NaN  NaN  NaN  4.0   
187 (1997)                                   NaN  NaN  2.0  NaN  NaN  NaN   
2 Days in the Valley (1996)                  NaN  NaN  NaN  NaN  NaN  NaN   
20,000 Leagues Under the Sea (1954)          3.0  NaN  NaN  NaN  NaN  NaN   
2001: A Space Odyssey (1968)                 4.0  NaN  NaN  NaN  4.0  5.0   
3 Ninjas: High Noon At Mega Mountain (1998)  NaN  1.0  NaN  NaN  NaN  NaN   
39 Steps, The (1935)                         NaN  NaN  NaN  NaN  NaN  NaN   

user_id                                      7    8    9    10   ...  934  

C:\Users\9mzx8\anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\9mzx8\anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


user_id
61     1.0
10     1.0
477    1.0
636    1.0
400    1.0
dtype: float64

協同過濾的優/缺點如下：

優點：協同過濾是集合眾人的意見進行推薦，比較有可信度。
缺點：冷啟動的問題，採用USER-USER 協同過濾法，遇到新顧客無購買或瀏覽記錄，無法計算與其他顧客的相似度。ITEM-ITEM 協同過濾法也是一樣，新商品也無交易記錄，，無法計算與其他商品的相似度。這時可以考慮使用『Content Based Filtering』。
以上程式是針對單一使用者瀏覽商品時，作出即時的推薦，如果，要執行所有的使用者的推薦清單，可能就要燒點錢，準備比較好的設備，採用平行計算，因為每一個相似性的計算都是可以獨立執行的。